In [1]:
import math
import numpy as np
import pandas as pd
import time
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RandomizedSearchCV

In [2]:
train_data = r'/home/hadoop/Downloads/P1 Data/Consumer_Complaints_train.csv'
test_data = r'/home/hadoop/Downloads/P1 Data/Consumer_Complaints_test_share.csv'

In [3]:
cct = pd.read_csv(train_data)
ccts = pd.read_csv(test_data)

In [4]:
ccts['Consumer disputed?'] = pd.Series([0]* len(ccts['Date received']))

In [5]:
cct['dtype'] = pd.Series(['train']*len(cct['Date received']))

In [6]:
ccts['dtype'] = pd.Series(['test']*len(ccts['Date received']))

In [7]:
ccts = pd.DataFrame(data = ccts)

In [5]:
ccts.tail()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Complaint ID
119601,2012-09-15,Credit card,NaN,Closing/Cancelling account,NaN,NaN,NaN,JPMorgan Chase & Co.,IL,60538,NaN,NaN,Web,2012-09-21,Closed with monetary relief,Yes,153482
119602,2016-07-21,Credit reporting,NaN,Credit reporting company's investigation,No notice of investigation status/result,On XXXX/XXXX/16 a dispute and validation of a ...,Company has responded to the consumer and the ...,Experian,CA,956XX,NaN,Consent provided,Web,2016-07-21,Closed with explanation,Yes,2023523
119603,2015-06-09,Debt collection,"Other (i.e. phone, health club, etc.)",Disclosure verification of debt,Not given enough info to verify debt,NaN,Company chooses not to provide a public response,"FAIR COLLECTIONS & OUTSOURCING, INC.",CA,90803,Servicemember,NaN,Phone,2015-06-12,Closed with explanation,Yes,1413678
119604,2014-02-26,Debt collection,Credit card,Disclosure verification of debt,Not given enough info to verify debt,NaN,NaN,Encore Capital Group,PA,17764,NaN,NaN,Phone,2014-02-27,Closed with explanation,Yes,732458
119605,2015-02-11,Debt collection,"Other (i.e. phone, health club, etc.)",Cont'd attempts collect debt not owed,Debt is not mine,NaN,NaN,AllianceOne Recievables Management,CA,92103,NaN,NaN,Web,2015-02-11,Closed with explanation,Yes,1235226


In [9]:
cct = pd.DataFrame(data = cct)

In [10]:
All_data = cct.append(ccts)

In [11]:
cct.shape

(478421, 19)

In [12]:
All_data

,Company,Company public response,Company response to consumer,Complaint ID,Consumer complaint narrative,Consumer consent provided?,Consumer disputed?,Date received,Date sent to company,Issue,Product,State,Sub-issue,Sub-product,Submitted via,Tags,Timely response?,ZIP code,dtype
0,Wells Fargo & Company,NaN,Closed with explanation,856103,NaN,NaN,No,2014-05-15,2014-05-16,Billing statement,Credit card,MI,NaN,NaN,Web,Older American,Yes,48342,train
1,Santander Bank US,NaN,Closed,1034666,NaN,NaN,No,2014-09-18,2014-09-24,"Making/receiving payments, sending money",Bank account or service,PA,NaN,(CD) Certificate of deposit,Referral,NaN,Yes,18042,train
2,Equifax,NaN,Closed with non-monetary relief,756363,NaN,NaN,No,2014-03-13,2014-04-03,Incorrect information on credit report,Credit reporting,CA,Account status,NaN,Referral,NaN,Yes,92427,train
3,U.S. Bancorp,Company chooses not to provide a public response,Closed with monetary relief,1474177,"My credit card statement from US Bank, XXXX. X...",Consent provided,No,2015-07-17,2015-07-17,Billing statement,Credit card,GA,NaN,NaN,Web,Older American,Yes,305XX,train
4,Bank of America,NaN,Closed with explanation,1132572,NaN,NaN,No,2014-11-20,2014-11-28,Transaction issue,Credit card,MA,NaN,NaN,Web,NaN,Yes,02127,train
5,Bank of America,NaN,Closed with explanation,913769,NaN,NaN,No,2014-06-26,2014-07-01,"Loan modification,collection,foreclosure",Mortgage,WI,NaN,Other mortgage,Referral,NaN,Yes,54313,train
6,Ocwen,NaN,Closed with explanation,163903,NaN,NaN,No,2012-09-28,2012-10-01,"Loan servicing, payments, escrow account",Mortgage,FL,NaN,Other mortgage,Referral,NaN,Yes,33168,train
7,Quicken Loans,NaN,Closed with explanation,1362945,I have a mortgage with Quicken Loans. From XXX...,Consent provided,Yes,2015-05-06,2015-05-06,"Loan modification,collection,foreclosure",Mortgage,GA,NaN,FHA mortgage,Web,NaN,Yes,300XX,train
8,Discover,NaN,Closed with monetary relief,331502,NaN,NaN,No,2013-02-25,2013-02-26,Credit card protection / Debt protection,Credit card,OH,NaN,NaN,Web,NaN,Yes,45640,train
9,Bank of America,Company has responded to the consumer and the ...,Closed with explanation,1855737,I put my settlement check in to bank of americ...,Consent provided,No,2016-03-30,2016-04-01,Deposits and withdrawals,Bank account or service,TX,NaN,Checking account,Web,NaN,Yes,787XX,train


In [13]:
All_data.isnull().sum()

Company                              0
Company public response         484859
Company response to consumer         0
Complaint ID                         0
Consumer complaint narrative    504376
Consumer consent provided?      428676
Consumer disputed?                   0
Date received                        0
Date sent to company                 0
Issue                                0
Product                              0
State                             4764
Sub-issue                       365685
Sub-product                     173225
Submitted via                        1
Tags                            513950
Timely response?                     0
ZIP code                          4774
dtype                                0
dtype: int64

In [14]:
All_data.shape

(598027, 19)

In [15]:
for col in ['Sub-product', 'Sub-issue','Consumer complaint narrative','Company public response','Tags'
           ,'Consumer consent provided?']:
    varname = col.replace('-','_').replace('?','').replace(' ','_')+ '_Nan'
    All_data[varname] = np.where(pd.isnull(All_data[col]),1,0)
    All_data.drop([col],1,inplace = True)
       

In [16]:
np.where(All_data['Submitted via'].isnull() == True)

(array([561027]),)

In [17]:
value = 'Web'
All_data['Submitted via'] = All_data['Submitted via'].fillna(value)

In [18]:
All_data.shape

(598027, 19)

In [19]:
All_data['Date received'] = pd.to_datetime(All_data['Date received'],infer_datetime_format=True)

In [20]:
All_data['Date sent to company'] = pd.to_datetime(All_data['Date sent to company'],infer_datetime_format=True)


In [21]:
All_data['day_diff'] = pd.to_numeric(All_data['Date sent to company']-All_data['Date received'])


In [22]:
All_data.drop(['Date received'],1, inplace = True)
All_data.drop(['Date sent to company'],1,inplace = True)



In [23]:
All_data.drop(['Company'],1, inplace = True)
All_data.drop(['ZIP code'],1,inplace = True)


In [24]:
for col in ['Product','Issue','State','Submitted via','Company response to consumer','Timely response?']:
    
    freq = All_data[col].value_counts()
    select_catgory = freq.index[freq> 200][:-1]
    
    print(col)
    
    for cat in select_catgory:
        name = col+'_'+cat
        All_data[name] = (All_data[col]== cat).astype(int)
    del All_data[col] 

Product
Issue
State
Submitted via
Company response to consumer
Timely response?


In [25]:
cct = All_data.iloc[0:478421,]

In [26]:
cct.tail()

,Complaint ID,Consumer disputed?,dtype,Sub_product_Nan,Sub_issue_Nan,Consumer_complaint_narrative_Nan,Company_public_response_Nan,Tags_Nan,Consumer_consent_provided_Nan,day_diff,...,Submitted via_Referral,Submitted via_Phone,Submitted via_Postal mail,Submitted via_Fax,Company response to consumer_Closed with explanation,Company response to consumer_Closed with non-monetary relief,Company response to consumer_Closed with monetary relief,Company response to consumer_Closed without relief,Company response to consumer_Closed,Timely response?_Yes
478416,1445488,No,train,0,1,1,1,1,1,86400000000000,...,0,1,0,0,1,0,0,0,0,1
478417,438011,No,train,1,1,1,1,1,1,86400000000000,...,0,1,0,0,1,0,0,0,0,1
478418,1020651,No,train,1,0,1,1,1,1,0,...,0,0,0,0,1,0,0,0,0,1
478419,157524,No,train,0,1,1,1,1,1,259200000000000,...,0,0,0,0,1,0,0,0,0,1
478420,1093195,No,train,0,1,1,1,0,1,0,...,0,0,0,0,0,1,0,0,0,1


In [27]:
ccts = All_data.iloc[478421:,]

In [28]:
ccts.shape

(119606, 157)

In [29]:
cct['Consumer disputed?'] = np.where(cct['Consumer disputed?']== 'Yes',1,0)

/home/hadoop/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [30]:
x = cct.drop(['Complaint ID','Consumer disputed?','dtype'],1)

In [31]:
y = cct['Consumer disputed?']

In [32]:
ccts = ccts.drop(['Complaint ID','Consumer disputed?','dtype'],1)

In [33]:
from sklearn.metrics import roc_auc_score

In [34]:
from sklearn import tree

In [35]:
dtree = tree.DecisionTreeClassifier(criterion="entropy",max_leaf_nodes=10,class_weight='balanced')

In [36]:
from sklearn.ensemble import AdaBoostClassifier

In [37]:
ada = AdaBoostClassifier(base_estimator = dtree, learning_rate = 0.1,n_estimators = 500)

In [38]:
final_model = ada.fit(x,y)

In [43]:
ccts1 = All_data.iloc[478421:,]

In [44]:
ccts1=ccts1.drop(['Consumer disputed?','dtype'],1)

In [45]:
prediction=np.where(final_model.predict(ccts1.drop(['Complaint ID'],1))==1,"Yes","No")
final_prediction=pd.DataFrame(list(zip(ccts1['Complaint ID'],list(prediction))),
                columns=['Complaint ID','Consumer disputed?'])

In [46]:
final_prediction

,Complaint ID,Consumer disputed?
0,675956,Yes
1,1858795,Yes
2,32637,Yes
3,1731374,No
4,501487,Yes
5,631565,Yes
6,1283963,Yes
7,1902970,No
8,1160691,No
9,359929,No


In [47]:
final_prediction.to_csv('Ajay_yadav_P1.csv',index=False)